In [1]:
import torch

In [2]:
a= torch.tensor(2.0, requires_grad= False)
b= torch.tensor(3.0, requires_grad= False)


## both a and b grad is false then  c grad will also be false so no grad calcialte


In [3]:
print("For a ")
print(f'data = {a.data}')
print(f'grad = {a.grad}')
print(f'is_leaf= {a.is_leaf}')
print(f'requires_grad= {a.requires_grad}')
print("\n\n")

print("For b ")
print(f'data = {b.data}')
print(f'grad = {b.grad}')
print(f'is_leaf= {b.is_leaf}')
print(f'requires_grad= {b.requires_grad}')

For a 
data = 2.0
grad = None
is_leaf= True
requires_grad= False



For b 
data = 3.0
grad = None
is_leaf= True
requires_grad= False


In [4]:
c = a * b 

print("For c ")
print(f'data = {c.data}')
print(f'grad = {c.grad}')
print(f'is_leaf= {c.is_leaf}')
print(f'requires_grad= {c.requires_grad}')


For c 
data = 6.0
grad = None
is_leaf= True
requires_grad= False


In [5]:
##if one grad is True then ouput grad will also be true 

In [6]:
a= torch.tensor(2.0, requires_grad= False)
b= torch.tensor(3.0, requires_grad= False)


print("For a ")
print(f'data = {a.data}')
print(f'grad = {a.grad}')
print(f'is_leaf= {a.is_leaf}')
print(f'requires_grad= {a.requires_grad}')
print("\n\n")

print("For b ")
print(f'data = {b.data}')
print(f'grad = {b.grad}')
print(f'is_leaf= {b.is_leaf}')
print(f'requires_grad= {b.requires_grad}')


c = a * b 

print("For c ")
print(f'data = {c.data}')
print(f'grad = {c.grad}')
print(f'is_leaf= {c.is_leaf}')
print(f'requires_grad= {c.requires_grad}')

For a 
data = 2.0
grad = None
is_leaf= True
requires_grad= False



For b 
data = 3.0
grad = None
is_leaf= True
requires_grad= False
For c 
data = 6.0
grad = None
is_leaf= True
requires_grad= False


## grad 

In [46]:
import torch

# Define tensors with requires_grad=True
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

# Define the computation
c = a * b

# Define a hook to print gradients (although in this case it's more relevant to show the result directly)
def c_hook(grad):
    print("Gradient for c:", grad)
    return grad + 2  # Just for demonstration; normally you'd return the gradient as-is

# Register the hook on tensor c
c.register_hook(c_hook)

# Perform backward pass to compute gradients
c.backward()

# Print gradients for tensors a and b
print("Gradient of a:", a.grad)
print("Gradient of b:", b.grad)


Gradient for c: tensor(1.)
Gradient of a: tensor(9.)
Gradient of b: tensor(6.)


## unnecessary usages of the grad 


In [54]:
a = torch.tensor(2., requires_grad= True).float()
b= torch.tensor(3., requires_grad= True).float()

#define the computation
c = a + b

d = torch.tensor(4., requires_grad= True).float()
def d_hook(grad):
    grad *= 100 

d.register_hook(d_hook)
e= c + d 
e.backward()

print('Gradient for a: ',a.grad)
print('Gradient for b:' , b.grad)
print('Gradien d:', d.grad)

Gradient for a:  tensor(100.)
Gradient for b: tensor(100.)
Gradien d: tensor(100.)


tensor(10., grad_fn=<AddBackward0>)

In [2]:

# Common Libraries
import random 
import numpy as np
import torch

# Image Augmentation 
from PIL import Image 
from image_classification import generate_dataset 

# Previously Defined Functions and Classes
from v1 import StepByStep 
from torch.utils.data import Dataset , DataLoader

# Data Utilities
from helpers import index_splitter, make_balanced_sampler
from torchvision.transforms import Compose,Normalize

#PyTorch Optimization and Loss Functions
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


#data geneartion 


images, labels= generate_dataset(img_size=10, n_images=100, binary=False, seed=17)

############## data preparation ######################### 
class TransformedTensorDataset(Dataset):
    def __init__(self, x, y, transformed=None):
        self.x = x
        self.y= y
        self.transfomed= transformed 

    def __getitem__(self, index):
        x= self.x[index]
        if self.transfomed:
            x=self.transfomed(x)
        return x, self.y[index] 

    def __len__(self, x):
        return len(self.x)
    

############# data preparation 2 #######################
# Builds tensors from numpy arrays BEFORE split
# Modifies the scale of pixel values from [0, 255] to [0, 1]
x_tensor = torch.as_tensor(images / 255).float()
y_tensor = torch.as_tensor(labels).long()

# Uses index_splitter to generate indices for training and
# validation sets
train_idx, val_idx = index_splitter(len(x_tensor), [80, 20])
# Uses indices to perform the split
x_train_tensor = x_tensor[train_idx]
y_train_tensor = y_tensor[train_idx]
x_val_tensor = x_tensor[val_idx]
y_val_tensor = y_tensor[val_idx]

# We're not doing any data augmentation now
train_composer = Compose([Normalize(mean=(.5,), std=(.5,))])
val_composer = Compose([Normalize(mean=(.5,), std=(.5,))])

# Uses custom dataset to apply composed transforms to each set
train_dataset = TransformedTensorDataset(x_train_tensor, y_train_tensor, transform=train_composer)
val_dataset = TransformedTensorDataset(x_val_tensor, y_val_tensor, transform=val_composer)

# Builds a weighted random sampler to handle imbalanced classes
sampler = make_balanced_sampler(y_train_tensor)

# Uses sampler in the training set to get a balanced data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)
####################### model configuration ##########################################################

torch.manual_seed(42)
model_cnn1= nn.Sequential()

n_channels= 1 
model_cnn1.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=n_channels, kernel_size=3))
model_cnn1.add_module('relu1',nn.ReLU())
model_cnn1.add_module('maxp1', nn.MaxPool2d(kernel_size=2))
#flattening : n channles *4 *4 

model_cnn1.add_module('flattern', nn.Flatten())

#clasification  #hidden layer
model_cnn1.add_module('fc1',  nn.Linear(in_features=n_channels*4*4, out_features= 10))
model_cnn1.add_module('relu2', nn.ReLU())

#output layer
model_cnn1.add_module('fc2', nn.Linear(in_features=10, out_features=3))


############ others parameters
lr= 0.3
multi_loss_fn= nn.CrossEntropyLoss(reducation='mean')
optimizer_cnn1= optim.SGD(model_cnn1.parameters(),lr=lr)



################# model training #########################
sbs_cnn1 = StepByStep(model_cnn1, multi_loss_fn, lr=lr)
sbs_cnn1.set_loaders(train_loader, val_loader) 

sbs_cnn1.train(20)

fig= sbs_cnn1.plot_losses()

IndexError: too many indices for tensor of dimension 4

In [3]:
# Builds tensors from numpy arrays BEFORE split
# Modifies the scale of pixel values from [0, 255] to [0, 1]
x_tensor = torch.as_tensor(images / 255).float()
y_tensor = torch.as_tensor(labels).long()

# Uses index_splitter to generate indices for training and
# validation sets
train_idx, val_idx = index_splitter(len(x_tensor), [80, 20])
# Uses indices to perform the split
x_train_tensor = x_tensor[train_idx]
y_train_tensor = y_tensor[train_idx]
x_val_tensor = x_tensor[val_idx]
y_val_tensor = y_tensor[val_idx]

# We're not doing any data augmentation now
train_composer = Compose([Normalize(mean=(.5,), std=(.5,))])
val_composer = Compose([Normalize(mean=(.5,), std=(.5,))])

# Uses custom dataset to apply composed transforms to each set
train_dataset = TransformedTensorDataset(x_train_tensor, y_train_tensor, transform=train_composer)
val_dataset = TransformedTensorDataset(x_val_tensor, y_val_tensor, transform=val_composer)

# Builds a weighted random sampler to handle imbalanced classes
sampler = make_balanced_sampler(y_train_tensor)

# Uses sampler in the training set to get a balanced data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)

IndexError: too many indices for tensor of dimension 4

In [6]:
from helpers import index_splitter
# Builds tensors from numpy arrays BEFORE split
# Modifies the scale of pixel values from [0, 255] to [0, 1]
x_tensor = torch.as_tensor(images / 255).float()
y_tensor = torch.as_tensor(labels).long()

# Uses index_splitter to generate indices for training and
# validation sets
train_idx, val_idx = index_splitter(len(x_tensor), [80, 20])
# Uses indices to perform the split
x_train_tensor = x_tensor[train_idx]
y_train_tensor = y_tensor[train_idx]
x_val_tensor = x_tensor[val_idx]
y_val_tensor = y_tensor[val_idx]

# We're not doing any data augmentation now
train_composer = Compose([Normalize(mean=(.5,), std=(.5,))])
val_composer = Compose([Normalize(mean=(.5,), std=(.5,))])

# Uses custom dataset to apply composed transforms to each set
train_dataset = TransformedTensorDataset(x_train_tensor, y_train_tensor, transform=train_composer)
val_dataset = TransformedTensorDataset(x_val_tensor, y_val_tensor, transform=val_composer)

# Builds a weighted random sampler to handle imbalanced classes
sampler = make_balanced_sampler(y_train_tensor)

# Uses sampler in the training set to get a balanced data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)

IndexError: too many indices for tensor of dimension 4

In [15]:
print(x_tensor.shape)  # Should be something like (N, C, H, W)
print(y_tensor.shape)  # Should be something like (N,)


torch.Size([100, 1, 10, 10])
torch.Size([100])


In [16]:
print(len(train_idx))  # Should be equal to number of samples in training set
print(len(val_idx))    # Should be equal to number of samples in validation set


80
20


In [17]:
print(len(train_idx))  # Should be equal to number of samples in training set
print(len(val_idx))    # Should be equal to number of samples in validation set


80
20


In [18]:
from helpers import index_splitter, make_balanced_sampler
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, Normalize
import torch

# Convert numpy arrays to PyTorch tensors and normalize pixel values
x_tensor = torch.as_tensor(images / 255.0).float()
y_tensor = torch.as_tensor(labels).long()

# Check tensor shapes
print(f"x_tensor shape: {x_tensor.shape}")
print(f"y_tensor shape: {y_tensor.shape}")

# Split indices into training and validation sets
train_idx, val_idx = index_splitter(len(x_tensor), [80, 20])

# Ensure indices are valid
print(f"train_idx length: {len(train_idx)}")
print(f"val_idx length: {len(val_idx)}")

# Apply the split to tensors
x_train_tensor = x_tensor[train_idx]
y_train_tensor = y_tensor[train_idx]
x_val_tensor = x_tensor[val_idx]
y_val_tensor = y_tensor[val_idx]

# Define normalization transforms for training and validation sets
transform = Compose([Normalize(mean=(0.5,), std=(0.5,))])

# Create datasets with transformations applied
train_dataset = TransformedTensorDataset(x_train_tensor, y_train_tensor, transform=transform)
val_dataset = TransformedTensorDataset(x_val_tensor, y_val_tensor, transform=transform)

# Create a weighted random sampler to handle class imbalance
sampler = make_balanced_sampler(y_train_tensor)

# Create data loaders for training and validation sets
train_loader = DataLoader(dataset=train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=16)


x_tensor shape: torch.Size([100, 1, 10, 10])
y_tensor shape: torch.Size([100])
train_idx length: 80
val_idx length: 20


IndexError: too many indices for tensor of dimension 4

In [20]:
y_train_tensor

tensor([2, 2, 0, 1, 0, 0, 1, 1, 0, 2, 1, 0, 1, 1, 2, 0, 1, 2, 0, 2, 1, 1, 2, 2,
        2, 1, 1, 1, 1, 2, 0, 2, 2, 1, 0, 2, 1, 0, 2, 0, 0, 2, 1, 0, 0, 1, 1, 2,
        2, 0, 2, 2, 2, 2, 1, 2, 2, 0, 1, 2, 1, 2, 0, 1, 0, 0, 2, 1, 0, 1, 0, 0,
        1, 2, 1, 0, 2, 1, 1, 0])

In [21]:

x_val_tensor

NameError: name 'x_val_tensor' is not defined